In [1]:
# импорт необходимых библиотек
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from typing import Union
import plotly.graph_objects as go
from scipy import stats
from math import asin
import numpy as np

In [5]:
#Расчёт effect_size для пропорций
def calc_propotion_effect_size(conv_1: int, nobs_1: int, conv_2: int, nobs_2: int):
    prob_1, prob_2 = conv_1 / nobs_1, conv_2 / nobs_2
    
    es_formula = 2  * asin(np.sqrt(prob_1)) - 2 * asin(np.sqrt(prob_2))
    es_import = proportion_effectsize(prob_1, prob_2)
    
    return es_formula, es_import

#Расчёт effect_size для непрерывных метрик
def calc_continuous_effect_size(mean_1: Union[float, int],
                                std_1: Union[float, int],
                                mean_2: Union[float, int],
                                std_2: Union[float, int],
                                nobs_1: int = 10_000,
                                nobs_2: int = 10_000):
    
    es_formula = (mean_1 - mean_2) / ((std_1**2 + std_2**2) / 2) ** 0.5
    es_import = effectsize_smd(mean_1, std_1, nobs_1, mean_2, std_2, nobs_2)[0]
    
    return es_formula, es_import 

In [6]:
#Расчёт минимально необходимой выборки * 2(для теста и контроля) для пропорций
def calc_sample_size_proportion(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)
#Расчёт минимально необходимой выборки * 2(для теста и контроля) для непрерывной метрики
def calc_sample_size_continuous(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)

### Задания на семинар
1. Мы хотим провести АБ-тест баннеров. Нынешняя конверсия в клик на баннер (CTR) равна 1.5%. Мы предполагаем, что с новыми баннерами CTR вырастет до 1.7%. Определите, сколько юзеров нам нужно отправить на каждую версию с уровнем доверия 95%


2. Мы хотим провести АБ-тест формы подтверждения заказа. Нынешняя конверсия в заказ равна 3%. Мы предполагаем, что конверсия вырастет на 0,1%. Определите, сколько юзеров нам нужно отправить на каждую версию с уровнем доверия 99% и уровнем мощности 90%


3. Дизайнер предложил добавить рекомендательную систему на этапе формирования корзины. Нынешний средний чек равен 2167 рублей, а стандартная ошибка (SD) равна 69. Мы предполагаем, что в новой версии средний чек вырастет 2180. Определите, сколько юзеров нам нужно отправить на каждую версию с уровнем доверия 95%


4. Вы решили сравниваем метрику деньги на юзера в двух группах. Размер выборки - 1000 элементов в каждой группе. 
Для проверки нормальности распределения на выборке в 1000 наблюдений применили , критерий Шапиро-Уилка и получили p-value, равный 0.00002, alpha = 5% то какой бы вывод мы могли сделать в данном случае?
В этом случае какой статистический критерий для проверки первоначальной гипотезы тут лучше всего подойдёт и почему ?


In [34]:
mu_control, mu_test = 2167, 2180
std_control, std_test = 69, 69# * 1.15

calc_continuous_effect_size(mean_1=mu_control, std_1=std_control, mean_2=mu_test, std_2=std_test) #-0.18840579710144928

(-0.17483621585265752, -0.17482965875688977)

In [35]:
effect_size = calc_continuous_effect_size(mean_1=mu_control, std_1=std_control, mean_2=mu_test, std_2=std_test)[1]
calc_sample_size_continuous(effect_size=effect_size, alpha=.05, beta=.2)

1029

In [23]:
conv_1, conv_2 = 3, 3.1
nobs_1, nobs_2 = 100, 100
prob_1, prob_2 = conv_1 / nobs_1, conv_2 / nobs_2 #0.015, 0.017
print(prob_1, prob_2)
calc_propotion_effect_size(conv_1=conv_1, nobs_1=nobs_1, conv_2=conv_2, nobs_2=nobs_2)

0.03 0.031


(-0.005815545699511593, -0.005815545699511593)

In [25]:
effect_size = calc_propotion_effect_size(conv_1=conv_1, nobs_1=nobs_1, conv_2=conv_2, nobs_2=nobs_2)[0]
calc_sample_size_proportion(effect_size=effect_size, alpha=.01, beta=.1)

1759803

In [43]:
size = 4999 * 4
normal_distr = stats.norm.rvs(size=size)
uniform_distr = stats.uniform.rvs(size=size)
expon_distr = stats.expon.rvs(size=size)

In [44]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=normal_distr, name='normal_distr',))
fig.add_trace(go.Histogram(x=uniform_distr, name='uniform_distr'))
fig.add_trace(go.Histogram(x=expon_distr, name='expon_distr'))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.5)
fig.show()

In [45]:
distr_name = list(
    zip(
        (normal_distr, uniform_distr, expon_distr),
        ('normal_distr', 'uniform_distr', 'expon_distr')
    )
)


In [46]:
for distr, name in distr_name:
    print(stats.shapiro(distr), name)

ShapiroResult(statistic=0.999933123588562, pvalue=0.9191898107528687) normal_distr
ShapiroResult(statistic=0.9545013308525085, pvalue=0.0) uniform_distr
ShapiroResult(statistic=0.8136374950408936, pvalue=0.0) expon_distr


D:\software\anaconda\lib\site-packages\scipy\stats\_morestats.py:1800: UserWarning:

p-value may not be accurate for N > 5000.



In [47]:
for distr, name in distr_name:
    print(stats.kstest(distr, 'norm'), name)

KstestResult(statistic=0.0041118871413895475, pvalue=0.8864207570634313) normal_distr
KstestResult(statistic=0.5000218872500166, pvalue=0.0) uniform_distr
KstestResult(statistic=0.5000274843580841, pvalue=0.0) expon_distr


In [48]:
for distr, name in distr_name:
    print(stats.kstest(distr, 'uniform'), name)

KstestResult(statistic=0.5001000200040008, pvalue=0.0) normal_distr
KstestResult(statistic=0.00437920575081413, pvalue=0.8361387511941701) uniform_distr
KstestResult(statistic=0.37157491208944227, pvalue=0.0) expon_distr


In [49]:
for distr, name in distr_name:
    print(stats.kstest(distr, 'expon'), name)

KstestResult(statistic=0.5001000200040008, pvalue=0.0) normal_distr
KstestResult(statistic=0.3678838232597974, pvalue=0.0) uniform_distr
KstestResult(statistic=0.0064006372484044505, pvalue=0.3841279811142134) expon_distr


In [52]:
#pip install distfit
from distfit import distfit

# Initialize for discrete distribution fitting
dfit = distfit(distr=[
    'norm', 'uniform', 'expon'
])

# Run distfit to and determine whether we can find the parameters from the data.
result = dfit.fit_transform(uniform_distr)

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm   ] [0.00 sec] [RSS: 3.58782] [loc=0.500 scale=0.289]
[distfit] >INFO> [uniform] [0.00 sec] [RSS: 0.0509925] [loc=0.000 scale=1.000]
[distfit] >INFO> [expon  ] [0.00 sec] [RSS: 6.83046] [loc=0.000 scale=0.500]
[distfit] >INFO> Compute confidence intervals [parametric]


In [53]:
result['model']

{'name': 'uniform',
 'score': 0.050992537065186166,
 'loc': 5.4863199773103055e-05,
 'scale': 0.9999332251200247,
 'arg': (),
 'params': (5.4863199773103055e-05, 0.9999332251200247),
 'model': <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x223f0d8fa60>,
 'bootstrap_score': 0,
 'bootstrap_pass': None,
 'color': '#e41a1c',
 'CII_min_alpha': 0.05005152445577434,
 'CII_max_alpha': 0.9499914270637965}

### Практическое задание
* Продакт на главной mail.ru решил протестировать в рекомендательной ленте контента вместо карточек со статьями видеоплеер с короткими видео. Нынешний таймспент на юзера в день в среднем равен 25 минут, а стандартная ошибка (SD) равна 15. Мы предполагаем, что в новой версии таймспент на юзера в день изменится на 10% от текущего значения. Средний трафик  20000 человек в день. Посчитайте сколько дней необходимо держать эксперимент при alpha = 5% и beta = 20%.


* Наша продуктовая команда в ecommerce магазине планирует запустить тест, направленный на ускорение загрузки сайта. Одна из основных метрик bounce rate в GA = 40%. Мы предполагаем, что при оптимизации сайта изменится минимум на 20% от текущего показателя.Средний трафик  4000 человек в день. Посчитайте сколько нам нужно дней держать эксперимент при alpha = 5% и beta = 20%
